# Reimplementation from scatch of YOLOv8 for Car Detection

This notebook contains the implementation of the Yolo-v8 architecture. We use it for car detection

## All the Imports

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
import os
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image, ImageDraw
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math

## Flags Setting

* `TRAINING`: when `True`, running the notebook will train the model and save it.

* `TRAINING_DEBUG`: used to debug the training phase, to understand if the model is learning properly. When both `TRAINING` and `TRAINING_DEBUG` are `False`, running the notebook will test the model.

* `V1_LOSS`: when `True`, a Yolo-v1-like loss is used: we modified the original loss adding a cls loss calculation for grid cell with no-object. If `V1_LOSS` is set to `False`, Yolo-v8 loss is used.

In [ ]:
TRAINING = True
TRAINING_DEBUG = False
V1_LOSS = True

## Constants
The constants above are used to index the encoded target and output in order to retrieve the corresponding quantities.

* `DELTA_X = 0`
* `DELTA_Y = 1`
* `WIDTH = 2`
* `HEIGHT = 3`
* `CONFIDENCE = 4`
* `CLASS = 5`

In [ ]:
DELTA_X = 0
DELTA_Y = 1
WIDTH = 2
HEIGHT = 3
CONFIDENCE = 4
CLASS = 5

## Utils Function

* `scale_to_range`: TODO
* `show_image_with_boxes`: TODO
* `from_grid_coordinates_to_bbox`: TODO


In [ ]:
def scale_to_range(bboxes, old_x=676, old_y=380, new_x = 128, new_y=128):
    x_scale = new_x / old_x
    y_scale = new_y / old_y
    
    for box in bboxes:
        box[0] = int(np.round(box[0]*x_scale))
        box[1] = int(np.round(box[1]*y_scale))
        box[2] = int(np.round(box[2]*x_scale))
        box[3] = int(np.round(box[3]*y_scale))
    return bboxes

In [ ]:
# WHAT TO DO WITH THE INPUT:
# STARTING FROM IMAGE_ID AND 4 PARAMETERS FOR BBOX
# DIVIDE THE IMAGE IN GRID SIZE (128 -> 16, 8 AND 4)
# CALCULATE THE CENTER OF THE OBJECTS IN THE IMAGE BY CALCULATING THE CENTER OF THE BOUNDING BOX -> WE HAVE IT
# CALCULATE THE CELL IN WHICH THE CENTER LIES: THAT IS THE CELL RESPONSIBLE OF CALCULATING THE BBOX
# TRANSFORM THE XYWH QUANTITIES IN DELTA_X DELTA_Y DELTA_W DELTA_H 
# ADD CONFIDENCE 100% AND CLASS PROBABILITY 1 (CAR) FOR THAT GRID
# PUT ALL ZEROS IN ALL THE OTHER CELLS IN WHICH WE DON'T HAVE OBJECTS
class YOLODataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.image_ids = self.annotations['image'].unique()
        
        
    def transform_in_grid_coordinates(self, x_center, y_center, width, height, size="large"):
        
        #print(f"la nostra bounding box è x_center = {x_center}, y_center = {y_center}, width = {width}, height = {height}\n")
    
        if size == "small":
            number_of_cells = 16
            n_pixel_per_grid = 8
        
        if size == "medium":
            number_of_cells = 8
            n_pixel_per_grid = 16
        
        if size == "large":
            number_of_cells = 4
            n_pixel_per_grid = 32
        
        i = 0
        for cell in range(number_of_cells * number_of_cells):
            #print(f"colonna = {cell}\n")
            #print(f"riga = {i}\n")
            
            if cell % number_of_cells  == 0:
                i += 1
            
            x_a = n_pixel_per_grid * (cell % number_of_cells)
            y_a = n_pixel_per_grid * (i-1) 
            #print(f"x_a = {x_a}, y_a = {y_a}\n")
            
            # if verifica se il centro ricade nella cella
            #print(f"controlliamo se il centro ricade tra x = {x_a} e x = {x_a + n_pixel_per_grid}...\n")
            if x_center >= x_a and x_center <= (x_a + n_pixel_per_grid):
                #print(f"controlliamo se il centro ricade tra x = {y_a} e x = {y_a + n_pixel_per_grid}...\n")
                if y_center >= y_a and y_center <= (y_a + n_pixel_per_grid):
                    #print("il centro ricade qua! Calcoliamo quantità!\n")
                    delta_x = (x_center - x_a) / n_pixel_per_grid
                    delta_y = (y_center - y_a) / n_pixel_per_grid
                    delta_width = width / 128
                    delta_height = height / 128 # 128 is the heigth and width of the image
                    #print(f"delta_x = {delta_x}, delta_y = {delta_y}, delta_width = {delta_width}, delta_height = {delta_height}")
                    confidence = 1 # 100% confidence that is a car
                    cl = 1 # 1 = car, 0 = nothing
                    column = cell
                    row = i-1
                    return delta_x, delta_y, delta_width, delta_height, confidence, cl, column%number_of_cells, row
                else:
                    continue
            else:
                  continue
            print("[ERROR]: there is a box but we have not found the cell in which it lies")
            

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]    
        
        # TEMPORARY: This is just an image in which, using grid size 4x4 we have two cars
        # in the same cell of the grid
        #image_id = "vid_4_26420.jpg"
        ### REMOVE AFTER DEBUGGING
        
        #print(f"image_id = {image_id}")
        
        img_path = os.path.join(self.img_dir, image_id)
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
                    
        boxes = self.annotations[self.annotations['image'] == image_id][['xmin', 'ymin', 'xmax', 'ymax']].values
        #print(f"ID: {image_id}, shape: {boxes.shape}\n")
        boxes = boxes.astype(float)
        
        boxes = scale_to_range(boxes)
        
        
        batch_size = 1 
        number_of_cells = 4
        label_encoding = torch.zeros(6, number_of_cells, number_of_cells)

        N_POS = 0
        for i in boxes:
            N_POS +=1
            
            xmin, ymin, xmax, ymax = i     
            x_center = int((xmin + xmax) / 2.0)
            y_center = int((ymin + ymax) / 2.0)
            width = int(xmax - xmin)
            height = int(ymax - ymin)
            delta_x, delta_y, delta_w, delta_h, confidence, cl, column, row = self.transform_in_grid_coordinates(x_center, y_center, width, height, size="large")
            
            box = [delta_x, delta_y, delta_w, delta_h, confidence, cl]
            
            label_encoding[0][row][column] = box[0]
            label_encoding[1][row][column] = box[1]
            label_encoding[2][row][column] = box[2]
            label_encoding[3][row][column] = box[3]
            label_encoding[4][row][column] = box[4]
            label_encoding[5][row][column] = box[5]
            
        return image, label_encoding, N_POS

    # DEBUG FUNCTION WHEN TRAINING_DEBUG = True
    # IT DOES THE SAME THING AS __get_item__ but having 
    # the image_id in input
    def get_item_debug(self, image_id):
        img_path = os.path.join(self.img_dir, image_id)
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        boxes = self.annotations[self.annotations['image'] == image_id][['xmin', 'ymin', 'xmax', 'ymax']].values
        #print(f"ID: {image_id}, shape: {boxes.shape}\n")
        boxes = boxes.astype(float)
        
        boxes = scale_to_range(boxes)
        
        batch_size = 1 
        number_of_cells = 4
        label_encoding = torch.zeros(6, number_of_cells, number_of_cells)

        N_POS = 0
        for i in boxes:
            N_POS +=1
            
            xmin, ymin, xmax, ymax = i     
            x_center = int((xmin + xmax) / 2.0)
            y_center = int((ymin + ymax) / 2.0)
            width = int(xmax - xmin)
            height = int(ymax - ymin)
            delta_x, delta_y, delta_w, delta_h, confidence, cl, column, row = self.transform_in_grid_coordinates(x_center, y_center, width, height, size="large")
            
            box = [delta_x, delta_y, delta_w, delta_h, confidence, cl]
            
            label_encoding[0][row][column] = box[0]
            label_encoding[1][row][column] = box[1]
            label_encoding[2][row][column] = box[2]
            label_encoding[3][row][column] = box[3]
            label_encoding[4][row][column] = box[4]
            label_encoding[5][row][column] = box[5]
            
        return image, label_encoding, N_POS


In [ ]:
def show_image_with_boxes(self, idx):
        image, boxes_list = self[idx]
        image_draw = ImageDraw.Draw(image)
        
        # Draw the bounding boxes
        for box in boxes_list:
            xmin, ymin, xmax, ymax = box
            image_draw.rectangle([xmin, ymin, xmax, ymax], outline="red", width=2)
        
        # Display the image
        plt.imshow(image)
        plt.axis("off")
        plt.show()

## Yolo-v8 Architecture

In [ ]:
import torch
import torch.nn as nn

class ConvBlock(nn.Module):
    def __init__(self, k, s, p, c=3, dim=64, mc=512, w=1,flag=1):
        super(ConvBlock, self).__init__()
        dim = int(dim)
        out = min(dim,mc)*w
        self.conv = nn.Conv2d(in_channels=c, out_channels=out, kernel_size=k, stride=s, padding=p)
        self.batch_norm = nn.BatchNorm2d(num_features=out)
        self.activation = nn.SiLU()
    
    def forward(self, x):
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.activation(x)
        return x
    

class Bottleneck(nn.Module):
    def __init__(self, k=3, s=1, p=1, c=3, dim=64, shortcut=True):
        super(Bottleneck, self).__init__()
        self.conv1 = ConvBlock(k,s,p,c,dim=dim,mc=512)
        self.conv2 = ConvBlock(k,s,p,c,dim=dim,mc=512)
        self.short = shortcut

    
    def forward(self, x):
        #print(x.shape)
        #print(self.conv1)
        #print(self.conv2)
        if self.short: 
            res = self.conv1(x)
            res = self.conv2(res)
            return x + res
        else: 
            res = self.conv1(x)
            res = self.conv2(x)
            return res 
            
class C2fBlock(nn.Module):
    def __init__(self, k=1, s=1, p=0, c=3, depth_multiple=1, shortcut=True, dim=64, mc=512, w=1, flag=1):
        super(C2fBlock, self).__init__() 
        self.conv1 = ConvBlock(k=1,s=1,p=0,c=c,dim=dim,mc=mc,w=w,flag=flag)
        half_c= int(dim / 2)
        if half_c == 512:
            half_dim = 512
        else:
            half_dim= int(dim / 2)
            
        if flag == 0: 
            self.bottlenecks = nn.ModuleList([Bottleneck(k=3,s=1,p=1,c=256,dim=256) for _ in range(depth_multiple)])
            new_input = int(512 / 2) * (depth_multiple + 2)
            self.conv2 = ConvBlock(k,s,p,c=new_input,dim=dim,mc=mc,w=w)
        else:
            self.bottlenecks = nn.ModuleList([Bottleneck(k=3,s=1,p=1,c=half_c,dim=half_dim) for _ in range(depth_multiple)])
            new_input = int(dim / 2) * (depth_multiple + 2)
            self.conv2 = ConvBlock(k,s,p,c=new_input,dim=dim,mc=mc,w=w)
    
    def forward(self, x):
        
        #print(f"x_input: {x.shape}")
        x = self.conv1(x)
              
       # print(f"x_conv1: {x.shape}")
        
        # Split the input tensor into two halves along the channel dimension
        x1, x2 = torch.split(x, x.size(1) // 2, dim=1)
        
        #print(f"x1: {x1.shape}, x2: {x2.shape}")
        
        
        # Process the other half (x2) through the bottlenecks
        bottleneck_outputs = []
        # append half of the input before processing
        bottleneck_outputs.append(x2.clone())
        for bott in self.bottlenecks:
            x2 = bott(x2)
            bottleneck_outputs.append(x2.clone())
            
        # this will concatenate half of the input before processing
        # and after each bottleneck processing  
        
        concatenated_bottleneck_outputs = torch.cat(bottleneck_outputs, dim=1)

        # add the other half
        x = torch.cat((x1, concatenated_bottleneck_outputs), dim=1)
        #print(f"x: {x.shape}, x1: {x1.shape}, conc: {concatenated_bottleneck_outputs.shape}")
        x = self.conv2(x)
        return x
    
class SPPF(nn.Module):
    def __init__(self, k=3, s=1, p=0, c=3, dim=64):
        super(SPPF, self).__init__() 
        
        self.conv1 = ConvBlock(k=k,s=s,p=0,c=c,dim=dim)
        self.pool1 = nn.MaxPool2d(kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=9, stride=1, padding=4)
        self.pool3 = nn.MaxPool2d(kernel_size=13, stride=1, padding=6)
        self.conv2 = ConvBlock(k=3,s=1,p=1,c=4*c,dim=dim)
        
    def forward(self, x):
        
        x = self.conv1(x)
        pool1 = self.pool1(x)
        pool2 = self.pool2(x)
        pool3 = self.pool3(x)
        #print(f"x: {x.shape}, pool1: {pool1.shape}, pool2: {pool2.shape}, pool3: {pool3.shape}")
        x = torch.cat([x, pool1, pool2, pool3], dim=1)
        #print(f"x_conc: {x.shape}")
        x = self.conv2(x)
        return x
    
class DetectBlock(nn.Module):
    def __init__(self, k=3, s=1, p=1, c=3, reg_max=1, nc=1, mc=512, w=1):
        super(DetectBlock, self).__init__()
        
        #reg_max = controlla la precisione della regression sulla bounding box 
        #nc = number of classes
        self.box_conv1 = ConvBlock(k,s,p,c=c,dim=64)
        self.box_conv2 = ConvBlock(k,s,p,c=64,dim=64)
        #self.box_conv3 = nn.Conv2d(in_channels=64, out_channels=4*reg_max, kernel_size=1, stride=1, padding=0)
        # 4 + 1 + 1 = 6 out_channels
        self.box_conv3 = nn.Conv2d(in_channels=64, out_channels=6, kernel_size=1, stride=1, padding=0)
        
        
        #self.class_conv1 = ConvBlock(k,s,p,c,dim=64)
        #self.class_conv2 = ConvBlock(k,s,p,c=64,dim=64)
        #self.class_conv3 = nn.Conv2d(in_channels=64, out_channels=nc, kernel_size=1, stride=1, padding=0)
        
    def forward(self, x): 
        
        if DEBUG_2:
            print("[Detect:]")
            print(f"Input: {x.shape}")
            print("\t [Conv]")
        ret1 = self.box_conv1(x)
        if DEBUG_2:
            print(f"Output: {ret1.shape}")
            print("\t [Conv]")
        ret1 = self.box_conv2(ret1)
        if DEBUG_2:
            print(f"Output: {ret1.shape}")
            print("\t [Conv2D]")
            print(self.box_conv3)
        ret1 = self.box_conv3(ret1)
        if DEBUG_2:
            print(f"Output: {ret1.shape}")
        
        #ret2 = self.class_conv1(x)
        #ret2 = self.class_conv2(ret2)
        #ret2 = self.class_conv3(ret2)
        
        return ret1#, ret2
               
class BackBone(nn.Module):
    def __init__(self, k=3, s=2, p=1, depth=1):
        super(BackBone, self).__init__()
        
        self.conv1 = ConvBlock(k,s,p)
        self.conv2 = ConvBlock(k,s,p, dim=128, c=64)
        self.c2f = C2fBlock(k=1,s=1,p=0,depth_multiple=3*depth,dim=128, c=128)
        self.conv3 = ConvBlock(k,s,p, dim=256, c=128)
        self.c2f_second = C2fBlock(k=1,s=1,p=0,depth_multiple=6*depth,dim=256, c=256)
        self.conv4 = ConvBlock(k,s,p,dim=512, c=256)
        self.c2f_third = C2fBlock(k=1,s=1,p=0,depth_multiple=6*depth,dim=512, c=512)
        self.conv5 = ConvBlock(k,s,p,dim=1024, c=512)
        self.c2f_last = C2fBlock(k=1,s=1,p=0,depth_multiple=3*depth,dim=min(1024,512), c=512)
        
    def forward(self, x):
        if DEBUG:
            print("[Layer: Conv 0]")
            print(f"Input Tensor Shape:  {x.shape}")
        x = self.conv1(x)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: Conv 1]")
            print(f"Input Tensor Shape:  {x.shape}")
        x = self.conv2(x)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: C2f 2]")
            print(f"Input Tensor Shape:  {x.shape}")
        x = self.c2f(x)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: Conv 3]")
            print(f"Input Tensor Shape:  {x.shape}")        
        x = self.conv3(x)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: C2f 4]")
            print(f"Input Tensor Shape:  {x.shape}") 
        x_first = self.c2f_second(x)
        if DEBUG:
            print(f"Output Tensor Shape: {x_first.shape}")

            print("[Layer: Conv 5]")
            print(f"Input Tensor Shape:  {x_first.shape}") 
        x = self.conv4(x_first)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: C2f 6]")
            print(f"Input Tensor Shape:  {x.shape}") 
        x_second = self.c2f_third(x)
        if DEBUG:
            print(f"Output Tensor Shape: {x_second.shape}")

            print("[Layer: Conv 7]")
            print(f"Input Tensor Shape:  {x_second.shape}")
        x = self.conv5(x_second)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: C2f 8]")
            print(f"Input Tensor Shape:  {x.shape}")
        x_last = self.c2f_last(x)
        if DEBUG:
            print(f"Output Tensor Shape: {x_last.shape}")
        
        return x_first, x_second, x_last
    
class Neck(nn.Module):
    def __init__(self, depth=1, scale=2):
        super(Neck, self).__init__()
        
        self.sppf = SPPF(k=1,dim=1024,c=512)
        self.upsample1 = nn.Upsample(scale_factor=2)
        self.upsample2 = nn.Upsample(scale_factor=2)
        self.c2f_block1 = C2fBlock(dim=512,c=1024,flag=1,shortcut=False)
        self.c2f_block2 = C2fBlock(dim=256,c=768,flag=1,shortcut=False) 
        self.c2f_block3 = C2fBlock(dim=512,c=768,flag=1,shortcut=False)
        self.c2f_block4 = C2fBlock(dim=1024,c=1024,flag=0,shortcut=False)
        self.conv1 = ConvBlock(k=3,s=2,p=1,dim=256,c=256)
        self.conv2 = ConvBlock(k=3,s=2,p=1,dim=512,c=512)
        
    def forward(self, x_first, x_second, x_last):
        
        if DEBUG:
            print("[Layer: SPPF 9]")
            print(f"Input Tensor Shape:  {x_last.shape}")
        out_sppf = self.sppf(x_last)
        if DEBUG:
            print(f"Output Tensor Shape: {out_sppf.shape}")

            print("[Layer: Upsample 10]")
            print(f"Input Tensor Shape:  {out_sppf.shape}")
        x = self.upsample1(out_sppf)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: Concat 11]")
            print(f"Input Tensor Shape:  {x.shape}, {x_second.shape}")
        x = torch.cat((x,x_second), dim=1)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: C2f 12]")
            print(f"Input Tensor Shape:  {x.shape}")
        conc1 = self.c2f_block1(x)
        if DEBUG:
            print(f"Output Tensor Shape: {conc1.shape}")

            print("[Layer: Upsample 13]")
            print(f"Input Tensor Shape:  {conc1.shape}")
        x = self.upsample2(conc1)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: Concat 14]")
            print(f"Input Tensor Shape:  {x.shape}, {x_first.shape}")
        x = torch.cat((x,x_first), dim=1)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: C2f 15]")
            print(f"Input Tensor Shape:  {x.shape}")
        det1 = self.c2f_block2(x)
        if DEBUG:
            print(f"Output Tensor Shape: {det1.shape}")

            print("[Layer: Conv 16]")
            print(f"Input Tensor Shape:  {det1.shape}")
        x = self.conv1(det1)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: Concat 17]")
            print(f"Input Tensor Shape:  {x.shape}, {conc1.shape}")
        x = torch.cat((x,conc1), dim=1)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: C2f 18]")
            print(f"Input Tensor Shape:  {x.shape}")
        det2 = self.c2f_block3(x)
        if DEBUG:
            print(f"Output Tensor Shape: {det2.shape}")

            print("[Layer: Conv 19]")
            print(f"Input Tensor Shape:  {det2.shape}")
        x = self.conv2(det2)
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: Concat 20]")
            print(f"Input Tensor Shape:  {x.shape}, {out_sppf.shape}")
        x = torch.cat((x,out_sppf), dim=1)
       
        if DEBUG:
            print(f"Output Tensor Shape: {x.shape}")

            print("[Layer: C2f 21]")
            print(f"Input Tensor Shape:  {x.shape}")
        det3 = self.c2f_block4(x)
        if DEBUG:
            print(f"Output Tensor Shape: {det3.shape}")
        
        return det1, det2, det3
    
class Head(nn.Module):
    def __init__(self):
        super(Head, self).__init__()
        
        #self.det1 = DetectBlock(c=256)
        #self.det2 = DetectBlock(c=512)
        self.det3 = DetectBlock(c=512)
        
    def forward(self, x3):
        #return self.det1(x1), self.det2(x2), self.det3(x3)
        return self.det3(x3)
    
class YOLO(nn.Module):
    def __init__(self):
        super(YOLO, self).__init__()
        self.h1 = BackBone()
        self.h2 = Neck()
        self.h3 = Head()
        
    def forward(self, x): 
        
        if DEBUG:
            print("---------- Backbone ----------")
            print("[Backbone Input]")
            print(f"Input Tensor Shape: {x.shape}")
        res1, res2, res3 = self.h1(x)
        if DEBUG:
            print("[Backbone Output]")
            print(f"Output Tensor Shape: \n\t\t     {res1.shape}, \n\t\t     {res2.shape}, \n\t\t     {res3.shape}")
            print("------------------------------")

        if DEBUG:
            print("---------- Neck ----------")
            print("[Neck Input]")
            print(f"Input Tensor Shape:  \n\t\t     {res1.shape}, \n\t\t     {res2.shape}, \n\t\t     {res3.shape}")
        det1, det2, det3 = self.h2(res1, res2, res3)
        if DEBUG:
            print("[Neck Output]")
            print(f"Output Tensor Shape: \n\t\t     {det1.shape}, \n\t\t     {det2.shape}, \n\t\t     {det3.shape}")
            print("------------------------------")

        if DEBUG:
            print("---------- Head ----------")
            print("[Head Input]")
            #print(f"Input Tensor Shape: \n\t\t     {det1.shape}, \n\t\t     {det2.shape}, \n\t\t     {det3.shape}")
            print(f"Input Tensor Shape: \n\t\t  {det3.shape}")
        det3 = self.h3(det3)
        if DEBUG:
            print("[Head Output]")
            #print(f"Output Tensor Bbox Loss: \n\t\t     {det1.shape}, \n\t\t     {det2.shape}, \n\t\t     {det3.shape}")
            print(f"Output Tensor Bbox Loss: \n\t\t     {det3.shape}")
            #print(f"Output Tensor Cls Loss: \n\t\t     {det1[1].shape}, \n\t\t     {det2[1].shape}, \n\t\t     {det3[1].shape}")
            print("------------------------------")

        #return det1, det2, det3
        m = torch.nn.Sigmoid()
        return m(det3)
    

## Random image from the training-set with its bbox 

In [ ]:
import os
import random
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

csv_file = '/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv'

img_dir = '/kaggle/input/car-object-detection/data/training_images'


# show a random image from the train-set with its bounding box 
def show_image(csv_file, img_dir):
    annotations = pd.read_csv(csv_file)
 
    img_dir = img_dir
    image_ids = annotations['image']
    
    random_number = random.randint(0, annotations.shape[0] - 1)
    
    random_number = 20
    
    print(f"image randomly chosen: {image_ids[random_number]}")
    
    image_id = image_ids[random_number]
    
    #img_path = os.path.join(img_dir, image_id)
    
    ## JUST FOR TESTING
    img_path = os.path.join(img_dir, "vid_4_11280.jpg")
    ## REMOVE LATER
    
    image = Image.open(img_path).convert("RGB")
    
    # Step 3: Convert the tensor to a NumPy array
    img_numpy = np.array(image)
    
    boxes = annotations[annotations['image'] == image_id][['xmin', 'ymin', 'xmax', 'ymax']].values
    boxes = boxes.astype(float)
      
    # Plot the image with bounding boxes
    fig, ax = plt.subplots(1)
    ax.imshow(img_numpy)
    
    for box in boxes:
        xmin, ymin, xmax, ymax = box  
        width = xmax - xmin
        height = ymax - ymin
        
        # Calculate the center point
        x_center = (xmin + xmax) / 2.0
        y_center = (ymin + ymax) / 2.0
        
        print(f"bbox = {[x_center, y_center, width, height]}")
        
        # Create a Rectangle patch
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
        
        # Plot the center point
        #ax.plot(x_center, y_center, 'ro')  # 'bo' is for blue circle marker
    
        # Add the patch to the Axes
        ax.add_patch(rect)
    
    plt.axis('off')  # Turn off axis
    plt.show()
    
show_image(csv_file, img_dir)

## Loss Calculation

### Yolo-v8 Loss Calculation

TODO: put image or latex code

#### Yolo-v8 Loss Hyperparameters

* `LAMBDA_BOX`: TODO
* `LAMBDA_CLS`: TODO

In [ ]:
LAMBDA_BOX = 5.5
LAMDBA_CLS = 1

In [ ]:
def compute_IoU(beta_output, beta_target):
    
    x_center_output = beta_output[0]
    y_center_output = beta_output[1]
    width_output = beta_output[2]
    heigth_output = beta_output[3]

    x_center_target = beta_target[0]
    y_center_target = beta_target[1]
    width_target = beta_target[2]
    heigth_target = beta_target[3]
    
    
    # first convert from the YOLO format to (x_min, y_min, x_max, y_max)
    xmin_output = (2*x_center_output - width_output) / 2
    ymin_output = (2*y_center_output - heigth_output) / 2
    xmax_output = (2*x_center_output + width_output) / 2 
    ymax_output = (2*y_center_output + heigth_output) / 2

    boxA = [xmin_output, ymin_output, xmax_output, ymax_output]

    xmin_target = (2*x_center_target - width_target) / 2
    ymin_target = (2*y_center_target - heigth_target) / 2
    xmax_target = (2*x_center_target + width_target) / 2 
    ymax_target = (2*y_center_target + heigth_target) / 2

    boxB = [xmin_target, ymin_target, xmax_target, ymax_target]
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou


def compute_box_loss(N_POS, indicator, beta_output, beta_target):
    x_center_output = beta_output[0]
    y_center_output = beta_output[1]
    width_output = beta_output[2]
    heigth_output = beta_output[3]

    x_center_target = beta_target[0]
    y_center_target = beta_target[1]
    width_target = beta_target[2]
    heigth_target = beta_target[3]

    q = compute_IoU(beta_output, beta_target)

    # Compute the L2 norm
    diff = ((x_center_target - x_center_output),(y_center_target - y_center_output))
    l2_norm = sum(x**2 for x in diff) ** 0.5
    
    #l2_norm = torch.linalg.norm(diff, ord=2)

    # RHO = 1 BECAUSE YES
    rho = 1

    # I am assuming v_xy is = v in the paper
    v = (4/math.pi**2) * (math.atan2(width_target, heigth_target) - math.atan2(width_output, heigth_output)) ** 2

    alpha = v / (1 - q)

    box_loss = (LAMBDA_BOX / N_POS) * indicator * ((1 - q) + (l2_norm / rho**2) + (alpha*v))
    
    return box_loss
    
def compute_cls_loss(N_POS, y_output, y_target):
    
    binary_cross_entropy = y_target*math.log(y_output) + (1 - y_target)*math.log(1 - y_output)
    
    # JUST 1 CLASS
    cls_loss = (LAMBDA_CLS / N_POS) * binary_cross_entropy
    
    return cls_loss

def compute_loss(N_POS, indicator, beta_output, beta_target, y_output, y_target):
    
    box_loss = compute_box_loss(N_POS, indicator, beta_output, beta_target)
    
    cls_loss = compute_cls_loss(N_POS, y_output, y_target)
    
    total_loss = box_loss + cls_loss 
    
    return total_loss

In [ ]:
def show_image_and_bbox(image, encoding_of_boxes, filename=None, training=False):
    
    image_np = None

    # if a filename is not provided, a 128 x 128 image is displayed
    if filename == None:
        image = image[0]

        if image.shape == torch.Size([3, 128, 128]):
            image_np = image.permute(1, 2, 0).numpy()

    # if a filename is provided, the 676 x 380 image is displayed
    else:
        img_path = os.path.join("/kaggle/input/car-object-detection/data/testing_images", filename)
    
        if training == True:
            img_path = os.path.join("/kaggle/input/car-object-detection/data/training_images", filename)
    
        image = Image.open(img_path).convert("RGB")
        image_np = np.array(image)
            
    # Plot the image with bounding boxes
    fig, ax = plt.subplots(1)
    ax.imshow(image_np)
    
    bboxes = from_grid_coordinate_to_bbox(encoding_of_boxes)

    for box in bboxes:
        
        #print(f"box = {box}")

        x_center = box[0]
        y_center = box[1]
        width = box[2]
        height = box[3]
        
        xmin = (2*x_center - width) / 2
        ymin = (2*y_center - height) / 2
        xmax = (2*x_center + width) / 2
        ymax = (2*y_center + height) / 2
        
        rect = None
        # if a filename is not provided, the image is 128 x 128 and so the box should be
        if filename == None:
            rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
        
        # if the image is 676 x 380, the rectangle must be scaled
        else:
            bbox = scale_to_range(bboxes=[[xmin, ymin, xmax, ymax]], old_x=128, old_y=128, new_x=676, new_y=380)
            bbox = bbox[0]
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            width = xmax - xmin
            height = ymax - ymin
            rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
        
        
        # Add the patch to the Axes
        ax.add_patch(rect)
        
        
        confidence_score = box[4]
        ax.text(xmin, ymin - 5, f'{confidence_score:.4f}', color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))

    
    plt.axis('off')  # Turn off axis
    plt.show()
    
    return 

In [ ]:
def from_grid_coordinate_to_bbox(output, size="large"):
        
        if size == "small":
            number_of_cells = 16
            n_pixel_per_grid = 8
        
        if size == "medium":
            number_of_cells = 8
            n_pixel_per_grid = 16
        
        if size == "large":
            number_of_cells = 4
            n_pixel_per_grid = 32
    
        bboxes = []
        for i in range(number_of_cells):
            for j in range(number_of_cells):
                
                if output[0][CLASS][i][j] >= 0.6:
                    delta_x = float(output[0][DELTA_X][i][j])
                    delta_y = float(output[0][DELTA_Y][i][j])
                    delta_w = float(output[0][WIDTH][i][j])
                    delta_h = float(output[0][HEIGHT][i][j])

                    x_a = n_pixel_per_grid * j
                    y_a = n_pixel_per_grid * i

                    x = delta_x * n_pixel_per_grid + x_a
                    y = delta_y * n_pixel_per_grid + y_a
                    w = delta_w * 128
                    h = delta_h * 128
                    confidence = float(output[0][CONFIDENCE][i][j])                 

                    bbox = [x, y, w, h, confidence]
                    print(f"bbox = {bbox}, class = {output[0][CLASS][i][j]}")

                    bboxes.append(bbox)

        return bboxes                 

### Yolo-v1 Loss Calculation

We have modified the classic Yolo-v1 loss, adding a cls loss calculation term also for no-object cell. Given position $(x,y)$ for each cell of the grid, we have that the loss is: 

$$
\displaystyle\sum_{x,y} (\mathcal{L}_{obj} + \lambda_{no-obj}\mathcal{L}_{no-obj})
$$

Where the loss function used when a cell contains an object is:
$$
\mathcal{L}_{obj} = \lambda_{coord}\mathcal{L}_{obj-box} + \mathcal{L}_{obj-conf} + \mathcal{L}_{obj-class} 
$$

With: 

$$
\mathcal{L}_{obj-box} = (\hat{\Delta{x}} - \Delta{x})^2 + (\hat{\Delta{y}} - \Delta{y})^2 + (\sqrt{\hat{\Delta{w}}} - \sqrt{\Delta{w}})^2 + (\sqrt{\hat{\Delta{h}}} - \sqrt{\Delta{h}})^2
$$

$$
\mathcal{L}_{obj-conf} = (\hat{{C}} - {C})^2 
$$

$$
\mathcal{L}_{obj-cls} = (\hat{p} - {p})^2 
$$
with $C = 1$ and $p = 1$

While the loss function used when a cell does not contain an object is:
$$
\mathcal{L}_{obj} = \mathcal{L}_{no-obj-conf} + \mathcal{L}_{no-obj-class}
$$
With analogous expression for the class and confidence losses, with the only difference of $C = 0$ and $p = 0$

#### Yolo-v1 Loss Hyperparameters

* `LAMBDA_COORD`: when > 1 it puts more importance on box parameters than cls and confidence
* `LAMBDA_NO_OBJ`: when < 1 it puts more importance on grid cells that contain object than cells which don't

In [ ]:
LAMBDA_COORD = 5
LAMBDA_NO_OBJ = 1

In [ ]:
def compute_V1_loss(target, output, obj_indicator, no_obj_indicator): 

    #print(f"\t[TARGET:] = \n \t delta_x = {[target[0][DELTA_X][i][j]]} \n \t delta_y = {[target[0][DELTA_Y][i][j]]} \n \t delta_w = {[target[0][WIDTH][i][j]]} \n \t delta_h = {[target[0][HEIGHT][i][j]]} \n \t confidence = {[target[0][CONFIDENCE][i][j]]} \n \t class = {[target[0][CLASS][i][j]]} ")
    #print(f"\t[OUTPUT:] = \n \t delta_x = {[output[0][DELTA_X][i][j]]} \n \t delta_y = {[output[0][DELTA_Y][i][j]]} \n \t delta_w = {[output[0][WIDTH][i][j]]} \n \t delta_h = {[output[0][HEIGHT][i][j]]} \n \t confidence = {[output[0][CONFIDENCE][i][j]]} \n \t class = {[output[0][CLASS][i][j]]} \n")
    # obj loss calculation
    #print(f"\t OBJ LOSS:\n")
    
    loss_box_obj_DELTA_X = (target[0][DELTA_X][i][j] - output[0][DELTA_X][i][j])**2
    loss_box_obj_DELTA_Y = (target[0][DELTA_Y][i][j] - output[0][DELTA_Y][i][j])**2
    loss_box_obj_WIDTH = (torch.sqrt(target[0][WIDTH][i][j]) - torch.sqrt(output[0][WIDTH][i][j]))**2
    loss_box_obj_HEIGHT = (torch.sqrt(target[0][HEIGHT][i][j]) - torch.sqrt(output[0][HEIGHT][i][j]))**2
    loss_box_obj = loss_box_obj_DELTA_X + loss_box_obj_DELTA_Y + loss_box_obj_WIDTH + loss_box_obj_HEIGHT
    
    #print(f"\t \t box loss delta_x = {loss_box_obj_DELTA_X}")
    #print(f"\t \t box loss delta_y = {loss_box_obj_DELTA_Y}")
    #print(f"\t \t box loss delta_w = {loss_box_obj_WIDTH}")
    #print(f"\t \t box loss delta_h = {loss_box_obj_HEIGHT}")
    #print(f"\t \t total box loss = {loss_box_obj}\n")
    
    loss_conf_obj = (target[0][CONFIDENCE][i][j] - output[0][CONFIDENCE][i][j])**2
    #print(f"loss_conf_obj = {loss_conf_obj}")
    
    #print(f"\t \t conf loss = {loss_conf_obj}\n")

    loss_cls_obj = (target[0][CLASS][i][j] - output[0][CLASS][i][j])**2
    #print(f"loss_cls_obj = {loss_cls_obj}")
    
    #print(f"\t \t cls loss = {loss_cls_obj}\n")

    cell_loss_obj = obj_indicator*((LAMBDA_COORD*loss_box_obj) + loss_conf_obj + loss_cls_obj)
    #print(f"cell_loss_obj = {cell_loss_obj}")

    #print(f"\t \t TOTAL OBJ LOSS = {cell_loss_obj}\n")

    #print(f"\t NO OBJ LOSS:\n")
    
    # no obj loss calculation 
    loss_conf_no_obj = (target[0][CONFIDENCE][i][j] - output[0][CONFIDENCE][i][j])**2
    #print(f"loss_conf_no_obj = {loss_conf_no_obj}")
    loss_cls_no_obj = (target[0][CLASS][i][j] - output[0][CLASS][i][j])**2
    #print(f"loss_cls_no_obj = {loss_cls_no_obj}")
    
    #print(f"\t \t conf loss = {loss_conf_no_obj}\n")

    cell_loss_no_obj = LAMBDA_NO_OBJ * no_obj_indicator * (loss_conf_no_obj + loss_cls_no_obj)
    #print(f"cell_loss_no_obj = {cell_loss_no_obj}")

    #print(f"\t \t TOTAL NO OBJ LOSS = {cell_loss_no_obj}\n")

    total_loss = cell_loss_obj + cell_loss_no_obj
    #print(f"cell_loss_obj = {cell_loss_obj} cell_loss_no_obj = {cell_loss_no_obj}")
    #print(f"total_loss = {total_loss}\n\n\n\n")
    
    #print(f"\t TOTAL LOSS OF THE CELL = {total_loss}")
    
    return total_loss


## Training Phase

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# DEBUGGING OPTIONS

# FIRST LEVEL OF GRANULARITY: IT SHOWS INPUT AND OUTPUT SIZES OF THE MAIN BLOCK OF THE ARCHITECTURE
DEBUG = False

# SECOND LEVEL OF GRANULARITY: IT SHOWS INPUT AND OUTPUT SIZES OF INTERNAL SUB-BLOCK TOO
DEBUG_2 = False

if TRAINING == True:

    # Initialize dataset and dataloader
    
    num_epochs = 700

    csv_file = '/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv'
    img_dir = '/kaggle/input/car-object-detection/data/training_images'
    transform = transforms.Compose([
        #data aug 
        # TRY MOSAIC AUGMENTATION
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])

    if DEBUG:
        print("========================================")
        print("        YOLOv8 Model Debug Output       ")
        print("========================================")
    model = YOLO()

    dataset = YOLODataset(csv_file=csv_file, img_dir=img_dir, transform=transform)
    #dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=1)


    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        
        for image, target, N_POS in dataloader:
            
            # just for debugging
            #andand_bbox(image, target)

            target = target.to(device)
            image = image.to(device)

            optimizer.zero_grad()

            # Predizioni del modello
            output = model(image)

            n_row = target.shape[2]
            n_column = target.shape[3]

            
            total_loss = 0
            for i in range(0, n_row):
                for j in range(0, n_column):

                    # computing indicators for obj and no_obj loss
                    if target[0][CONFIDENCE][i][j] == 0: # -> there is no object!
                        obj_indicator = 0
                        no_obj_indicator = 1
                    else:
                        obj_indicator = 1
                        no_obj_indicator = 0

                    if V1_LOSS == True:
                        total_loss += compute_V1_loss(target, output, obj_indicator, no_obj_indicator)
                    else:
                        beta_output = output[0][:4][i][j]
                        beta_target = target[0][:4][i][j]
                        y_output = output[0][5][i][j]
                        y_target = target[0][5][i][j]
                        total_loss += compute_V8_loss(N_POS, obj_indicator, beta_output, beta_target, y_output, y_target)

            total_loss.backward()
            optimizer.step()

            epoch_loss += total_loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(dataloader)}")

### Debug Training

In [ ]:
if TRAINING_DEBUG == True:    
    
    # Initialize dataset and dataloader

    num_epochs = 20

    csv_file = '/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv'
    img_dir = '/kaggle/input/car-object-detection/data/training_images'
    transform = transforms.Compose([
        #data aug 
        # TRY MOSAIC AUGMENTATION
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])

    if DEBUG:
        print("========================================")
        print("        YOLOv8 Model Debug Output       ")
        print("========================================")
    model = YOLO()

    dataset = YOLODataset(csv_file=csv_file, img_dir=img_dir, transform=transform)
    

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    model.to(device)

    print("STARTING....\n")
    for epoch in range(num_epochs):
        
        model.train()
        epoch_loss = 0.0
        
        # prendi l'immagine 
        image, target, N_POS = dataset.get_item_debug("vid_4_28220.jpg")
        
        # to add the batch_size dimension as the dataloader does
        image = image.unsqueeze(0)
        target = target.unsqueeze(0)
        
        if epoch == 0:
            #just for debugging
            show_image_and_bbox(image, target)
            print(f"[TARGET:] = \n {target}\n")

        target = target.to(device)
        image = image.to(device)
        

        optimizer.zero_grad()

        # Predizioni del modello
        output = model(image)
        
        print(f"[OUTPUT:] = \n {output}\n")
        
        n_row = target.shape[2]
        n_column = target.shape[3]


        print("CALCULATE LOSS BETWEEN [TARGET] AND [OUTPUT]\n")
        total_loss = 0
        for i in range(0, n_row):
            for j in range(0, n_column):
                print(f"\tGrid Cell: row {i} column {j}\n")

                # computing indicators for obj and no_obj loss
                if target[0][CONFIDENCE][i][j] == 0: # -> there is no object!
                    obj_indicator = 0
                    no_obj_indicator = 1
                else:
                    obj_indicator = 1
                    no_obj_indicator = 0

                if V1_LOSS == True:
                    total_loss += compute_V1_loss(target, output, obj_indicator, no_obj_indicator)
                    print("\n")
                else:
                    beta_output = output[0][:4][i][j]
                    beta_target = target[0][:4][i][j]
                    y_output = output[0][5][i][j]
                    y_target = target[0][5][i][j]
                    total_loss += compute_V8_loss(N_POS, obj_indicator, beta_output, beta_target, y_output, y_target)

        total_loss.backward()
        optimizer.step()

        epoch_loss += total_loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / 1}")
        show_image_and_bbox(image.cpu(), output)
        print("\n")

## Save the Model after Training

In [ ]:
if TRAINING == True and TRAINING_DEBUG == False:
    torch.save(model.state_dict(), '/kaggle/working/yolov8_new_700_epoch.pth')
    print("model saved")

In [ ]:
# This section is only for train debugging
if TRAINING_DEBUG == True and TRAINING == False:
    !mkdir /kaggle/working/debugging
    torch.save(model.state_dict(), '/kaggle/working/debugging/yolov8_new_100_epoch_DEBUGGING.pht')
    print("debug model saved")

## Testing

### Load the Model 

In [ ]:
if TRAINING == False and TRAINING_DEBUG == False:
    model = YOLO();
    model_load_path = '/kaggle/input/yolo_v8_100_epoch/pytorch/default/1/yolov8_new_100_epoch.pth'
    model.load_state_dict(torch.load(model_load_path));
    model.to(device);
    model.eval();  # Set the model to evaluation mode
    print("model loaded")

### Test the Model

In [ ]:
if TRAINING == False and TRAINING_DEBUG == False:

    # Testing 
    test_transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])

    test_img_dir = '/kaggle/input/car-object-detection/data/testing_images'
    file_list = os.listdir(test_img_dir)
    TEST_SET_SIZE = len(file_list)

    random_number = random.randint(0, TEST_SET_SIZE)

    filename = None
    
    image = None
    for i in range(TEST_SET_SIZE):
        if i == random_number:
            filename = file_list[i]
            print(f"filename = {filename} randomly chosen")

            img_path = os.path.join(test_img_dir, filename)

            image = Image.open(img_path).convert("RGB")

            break 
            

    image = test_transform(image)
    image = image.unsqueeze(0)
    image = image.to(device)

    model.eval()

    with torch.no_grad():
        
        out = model(image)
        
        # giving filename to the function, it displays the result in 676 x 380
        # without it, it displays it in 128 x 128
        
        # if training = true you want to use images from the training set
        show_image_and_bbox(image.cpu(), out, filename, training=False)

## Post Processing

### Put the result in a directory for further processing

In [ ]:
# Put jpg in kaggle/working/results

### From a sequence of JPG to a video 

In [ ]:
"""

def create_video_from_images(image_folder, output_video_path, frame_rate=30):
    # Ottieni una lista di tutti i file nella cartella delle immagini
    images = [img for img in os.listdir(image_folder) if img.endswith((".jpg"))]
    
    # Se la lista delle immagini è vuota, esci
    if not images:
        print("La cartella delle immagini è vuota.")
        return
    
    # Leggi la prima immagine per ottenere le dimensioni del video
    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, layers = frame.shape

    # Definisci il codec e crea l'oggetto VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))
    
    # Aggiungi tutte le immagini al video
    for image in images:
        image_path = os.path.join(image_folder, image)
        frame = cv2.imread(image_path)
        video.write(frame)
    
    # Rilascia l'oggetto VideoWriter
    video.release()
    print(f"Video creato con successo: {output_video_path}")

# Esempio di utilizzo
image_folder = '/kaggle/input/car-object-detection/data/testing_images'
output_video_path = '/kaggle/working/video.mp4'
create_video_from_images(image_folder, output_video_path, frame_rate=30)


"""

### From Video to sequence of JPG

In [ ]:
"""

def create_images_from_video(video_path, output_dir, ):

    # Create the directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    frame_count = 0

    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # If no frame is read (end of the video), break the loop
        if not ret:
            break

        # Construct the file name for the image
        image_path = os.path.join(output_dir, f"frame_{frame_count}.jpg")

        # Save the frame as a JPEG file
        cv2.imwrite(image_path, frame)

        # Increment the frame count
        frame_count += 1

    # Release the video capture object
    cap.release()

    print(f'Extracted {frame_count} frames and saved to {output_dir}')
    
    
"""